# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!pip install citipy

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys

sys.path.append('../')
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
df_weather_data = pd.read_csv("../Weather/weather_data.csv")
df_weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Poum,-20.2333,164.0167,26.65,69,9,2.07,NC,1615779404
1,Ushuaia,-54.8000,-68.3000,9.00,71,75,3.60,AR,1615779268
2,Huanan,46.2167,130.5167,8.87,52,100,7.73,CN,1615779413
3,Kahului,20.8947,-156.4700,25.00,94,40,2.57,US,1615779194
4,Ngaoundéré,7.3167,13.5833,18.57,86,100,1.87,CM,1615779414


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with the API Key
gmaps.configure(api_key=g_key)

In [5]:
# Store 'Lat' and 'Lng' into  locations 
locations = df_weather_data[["Lat", "Lng"]].astype(float)

# Convert humidity as float and store
humidity = df_weather_data["Humidity"].astype(float)

In [6]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
def ftoc(fahrenheit):
    return ((fahrenheit - 32.0)*5/9)

def mphtompers(mph):
    return (mph/2.237)

In [8]:
# Filtering out all the max temperature between 70 and 80 degree Fahrenheit 
df_filtered = df_weather_data[df_weather_data['Max Temp'] < ftoc(80.0)]
df_filtered = df_filtered[df_filtered['Max Temp'] > ftoc(70.0)]

# Filtering so we only have cities with less than 10mph wind speed
df_filtered = df_filtered[df_filtered['Wind Speed'] < mphtompers(10.0)]

# Filtering for zero cloudiness cities only
df_filtered = df_filtered[df_filtered['Cloudiness'] == 0]

df_filtered

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,Pisco,-13.7000,-76.2167,25.00,65,0,1.54,PE,1615779276
63,Manacapuru,-3.2997,-60.6206,24.00,94,0,0.51,BR,1615779431
96,São Filipe,14.8961,-24.4956,21.49,73,0,1.82,CV,1615779440
112,Caravelas,-17.7125,-39.2481,22.98,84,0,1.88,BR,1615779281
149,Saquarema,-22.9200,-42.5103,25.00,100,0,3.09,BR,1615779455
151,Lakes Entrance,-37.8811,147.9810,23.33,48,0,0.89,AU,1615779375
274,Nioro,13.3500,-15.7500,21.42,22,0,2.36,GM,1615779489
302,São Filipe,14.8961,-24.4956,21.49,73,0,1.82,CV,1615779440
364,Pokhara,28.2333,83.9833,25.76,28,0,2.49,NP,1615779515
545,São Filipe,14.8961,-24.4956,21.49,73,0,1.82,CV,1615779440


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# set up a parameters dictionary
params = {
    "keyword": "Hotel",
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

df_filtered['Hotel'] = ""

In [10]:
# use iterrows to iterate through pandas dataframe
for index, row in df_filtered.iterrows():

    # get target location from df
    target_loc = f"{row['Lat']},{row['Lng']}"

    # add keyword to params dict
    params['location'] = target_loc

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {row['City']} is {results[0]['name']}.")
        
        df_filtered.loc[index, 'Hotel'] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 7: Pisco.
Closest hotel in Pisco is HOSTAL MIRAMAR.
------------
Retrieving Results for Index 63: Manacapuru.
Closest hotel in Manacapuru is Hotel Amazon Experience.
------------
Retrieving Results for Index 96: São Filipe.
Closest hotel in São Filipe is Hotel Xaguate.
------------
Retrieving Results for Index 112: Caravelas.
Closest hotel in Caravelas is Mar Doce de Abrolhos.
------------
Retrieving Results for Index 149: Saquarema.
Closest hotel in Saquarema is Hotel Maasai.
------------
Retrieving Results for Index 151: Lakes Entrance.
Closest hotel in Lakes Entrance is The Esplanade Resort & Spa.
------------
Retrieving Results for Index 274: Nioro.
Missing field/result... skipping.
------------
Retrieving Results for Index 302: São Filipe.
Closest hotel in São Filipe is Hotel Xaguate.
------------
Retrieving Results for Index 364: Pokhara.
Closest hotel in Pokhara is Hotel Crown Himalayas.
------------
Retrieving Results for Index 545: São Filipe.
Clos

In [11]:
# Cleaning up filtered dataframe so index is correct and dropping anything that does not have Hotel results
df_filtered = df_filtered.reset_index(drop=True)
df_filtered = df_filtered.drop(df_filtered[df_filtered["Hotel"]==""].index)
df_filtered = df_filtered.reset_index(drop=True)
df_filtered

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
0,Pisco,-13.7000,-76.2167,25.00,65,0,1.54,PE,1615779276,HOSTAL MIRAMAR
1,Manacapuru,-3.2997,-60.6206,24.00,94,0,0.51,BR,1615779431,Hotel Amazon Experience
2,São Filipe,14.8961,-24.4956,21.49,73,0,1.82,CV,1615779440,Hotel Xaguate
3,Caravelas,-17.7125,-39.2481,22.98,84,0,1.88,BR,1615779281,Mar Doce de Abrolhos
4,Saquarema,-22.9200,-42.5103,25.00,100,0,3.09,BR,1615779455,Hotel Maasai
5,Lakes Entrance,-37.8811,147.9810,23.33,48,0,0.89,AU,1615779375,The Esplanade Resort & Spa
6,São Filipe,14.8961,-24.4956,21.49,73,0,1.82,CV,1615779440,Hotel Xaguate
7,Pokhara,28.2333,83.9833,25.76,28,0,2.49,NP,1615779515,Hotel Crown Himalayas
8,São Filipe,14.8961,-24.4956,21.49,73,0,1.82,CV,1615779440,Hotel Xaguate
9,Tigre,-34.4260,-58.5796,24.00,60,0,4.12,AR,1615779574,Villa San Gimignano


In [12]:

# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in df_filtered.iterrows()]
locations = df_filtered[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(
    locations,
    info_box_content= hotel_info
)

# Display figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))